### Data Processing

This notebook includes the data preparation work for convertible bond research project.

**The main purpose of this notebook is**:
1. explore and get familiar with the raw datasets
2. reorganize useful information and combine them into structured dataframe or dictionary data type
3. check the data quality, supplement extra data if necessary and create necessary features for later usage

**Input**:
+ `data`: includes basic information and trading data of convertible bond.
+ `stock_data`: includes trading data of stock.
+ `index_data`: includes historical stock index members.

**Output**:
+ `cbond_info.csv`: basic information of convertible bond
+ `cbond_price.csv`: trading data of convertible bond
+ `stock_data.csv`: trading data of stock
+ several pickle files used in backtesting (for the purpose of saving time)

In [1]:
import os
import re
import math
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt 

plt.style.use('seaborn')
pd.set_option('display.max_columns',None)

In [2]:
data_dir = '../../../../export/scratch/for_yifan'
output_dir =  '../../../../export/scratch/for_yifan/research/'

**Data Processing for `data`**

In [3]:
cbond_files = os.listdir(os.path.join(data_dir, 'data'))
cbond_files

['ASHAREDESCRIPTION_202103031002.csv',
 'ASHAREDESCRIPTION',
 'CCBONDREDEMPTIONPRICERATE_202010221230.csv',
 'CCBONDREPURCHASEPRICERATE_202010221230.csv',
 'CCBONDISSUANCE_202010221230.csv',
 'CBONDPRICES_202010221134.csv',
 'CBONDCONVPRICE_202010221103.csv']

In [4]:
# key-filename dict
cbond_file_map = {
    'comp_info': 'ASHAREDESCRIPTION_202103031002.csv',
    'new_comp_info':'ASHAREDESCRIPTION',
    'issuance':'CCBONDISSUANCE_202010221230.csv',
    'redemption':'CCBONDREDEMPTIONPRICERATE_202010221230.csv',
    'repurchase': 'CCBONDREPURCHASEPRICERATE_202010221230.csv',
    'convprice':'CBONDCONVPRICE_202010221103.csv',
    'price': 'CBONDPRICES_202010221134.csv',
}

+ Company information 

In [5]:
comp_info_raw = pd.read_csv(os.path.join(data_dir, 'data', cbond_file_map['new_comp_info']), delimiter="|")
comp_info_raw.head(3)

,OBJECT_ID,S_INFO_WINDCODE,S_INFO_CODE,S_INFO_NAME,S_INFO_COMPNAME,S_INFO_COMPNAMEENG,S_INFO_ISINCODE,S_INFO_EXCHMARKET,S_INFO_LISTBOARD,S_INFO_LISTDATE,S_INFO_DELISTDATE,S_INFO_SEDOLCODE,CRNCY_CODE,S_INFO_PINYIN,S_INFO_LISTBOARDNAME,IS_SHSC,S_INFO_COMPCODE,OPDATE,OPMODE
0,{0000B5D4-B786-4B03-A7B0-50E3767ABF84},600373.SH,600373,中文传媒,中文天地出版传媒集团股份有限公司,Chinese Universe Publishing and Media Group Co...,CNE0000019X4,SSE,434004000,20020304.0,NaN,NaN,CNY,zwcm,主板,1.0,1600373,2015-09-09 09:07:23,1
1,{001EC435-194A-2377-E050-C80A10010479},A14629.SZ,A14629,中科创新(IPO终止),武汉中科创新技术股份有限公司,"Wuhan Zhongke Innovation Technology Co., Ltd.",NaN,SZSE,434001000,NaN,NaN,NaN,CNY,zkcx,创业板,0.0,RX7ADiN6pM,2020-09-07 16:27:02,1
2,{001EE265-3F28-DD72-E050-C80A100105DE},300557.SZ,300557,理工光科,武汉理工光科股份有限公司,Wuhan Ligong Guangke Co.Ltd.,CNE100002DC1,SZSE,434001000,20161101.0,NaN,NaN,CNY,lggk,创业板,0.0,1uF6BE2FE5,2020-08-21 16:32:24,1


In [6]:
comp_info = comp_info_raw[['S_INFO_COMPCODE', 'S_INFO_WINDCODE', 'S_INFO_NAME', 
                                'S_INFO_EXCHMARKET', 'S_INFO_LISTBOARDNAME']].copy()

comp_info.rename(columns = {'S_INFO_COMPCODE': 'comp_code',
                                 'S_INFO_WINDCODE': 'stock_ticker',
                                 'S_INFO_NAME':'name',
                                 'S_INFO_EXCHMARKET':'market',
                                 'S_INFO_LISTBOARDNAME':'listboard'}, inplace=True)
comp_info.head()

,comp_code,stock_ticker,name,market,listboard
0,1600373,600373.SH,中文传媒,SSE,主板
1,RX7ADiN6pM,A14629.SZ,中科创新(IPO终止),SZSE,创业板
2,1uF6BE2FE5,300557.SZ,理工光科,SZSE,创业板
3,1000416,000416.SZ,民生控股,SZSE,主板
4,2o557A92BF,002156.SZ,通富微电,SZSE,主板


In [7]:
# check nan data
comp_info.isnull().sum().sum()

0

In [8]:
# save company info to csv
comp_info.to_csv(os.path.join(output_dir, 'comp_info.csv'), index=False)

+ Issurance (发行条款)

In [9]:
issuance_raw = pd.read_csv(os.path.join(data_dir, 'data', cbond_file_map['issuance']), delimiter="|")
issuance_raw.head(3)

,OBJECT_ID,S_INFO_WINDCODE,CRNCY_CODE,ANN_DT,CB_INFO_PREPLANDATE,CB_INFO_SMTGANNCEDATE,CB_ISSUE_ANNCELSTDATE,CB_INFO_LISTEDDATE,CB_INFO_LISTDATE,CB_INFO_ISSEPARATION,CB_INFO_DISTRIBUTO,CB_INFO_RECOMMENDER,CB_CLAUSE_ISCHAINTEREST,CB_CLAUSE_ISCOMINTEREST,CB_CLAUSE_COMINTEREST,CB_CLAUSE_COMINTERESTITEM,CB_CLAUSE_CONVERSIONITEM,CB_CLAUSE_CONVCHANGEITEM,CB_CLAUSE_CONVMONTH,CB_CLAUSE_INICONVPRICE,CB_CLAUSE_INICONVPROPORTION,CB_CLAUSE_CALLITEM,CB_CLAUSE_RESET_ITEM,CB_CLAUSE_RESETITEM,CB_CLAUSE_RATIONITEM,CB_LIST_PASSDATE,CB_LIST_PERMITDATE,CB_LIST_ANNOUNCEDATE,CB_LIST_ANNOCEDATE,CB_LIST_TYPE,CB_LIST_FEE,CB_LIST_RATIONDATE,CB_LIST_RATIONCHKINDATE,CB_LIST_RATIONPAYMTDATE,CB_LIST_RATIONCODE,CB_LIST_RATIONNAME,CB_LIST_RATIONPRICE,CB_LIST_RATIONRATIODE,CB_LIST_RATIONRATIOMO,CB_LIST_RATIONVOL,CB_LIST_ORIGINALS,CB_LIST_DTONL,CB_LIST_PCHASECODEONL,CB_LIST_PCHNAMEONL,CB_LIST_PCHPRICEONL,CB_LIST_ISSUEVOLONL,CB_LIST_CODEONL,CB_LIST_EXCESSPCHONL,CB_RESULT_EFSUBSCRPOFF,CB_RESULT_SUCRATEOFF,CB_LIST_DATEINSTOFF,CB_LIST_VOLINSTOFF,CB_RESULT_SUCRATEON,CB_LIST_EFFECTPCHVOLOFF,CB_LIST_EFFPCHOF,CB_LIST_SUCRATEOFF,CB_LIST_PRERATIONVOL,S_INFO_COMPCODE,CB_LIST_ISSUESIZE,CB_LIST_ISSUEQUANTITY,SEC_ID,MINUNLINE_NO,DEPUNLINE_RATIO,MAXUNLINE_NO,UNLINE_UD,IS_CONVERTIBLE_BONDS,MINUNLINE_PUBLIC,MAXUNLINE_PUBLIC,B_INFO_TERM_YEAR_,B_INFO_INTERESTTYPE,B_INFO_COUPONRATE,B_INFO_INTERESTFREQUENCY,CB_RESULT_SUCRATEON2,B_INFO_COUPONTXT,S_RATIOANNCEDATE,S_RATIODATE,FIRSTPRELANDATE,OPDATE,OPMODE
0,{00333E40-AABF-11E8-8350-C03FD5023FE3},NaN,CNY,20180915,20180829.0,20180915.0,NaN,NaN,2,0.0,NaN,NaN,NaN,1.0,NaN,在本次发行的可转债期满后五个交易日内，公司将赎回全部未转股的可转债，具体赎回价格由股东大会授...,本次发行可转债的初始转股价格不低于本次可转债募集说明书公告日前二十个交易日公司股票交易均价（...,在本次发行之后，当公司因派送股票股利、转增股本、增发新股或配股、派送现金股利等情况（不包括因...,本次可转债转股期自可转债发行结束之日满六个月后的第一个交易日起至可转债到期日止。,NaN,NaN,本次发行的可转债最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于当期转股价...,转股期内，如果公司股票连续三十个交易日中至少有十五个交易日的收盘价不低于当期转股价格的130...,在本次发行的可转债存续期间，当公司股票在任意连续二十个交易日中至少有十个交易日的收盘价低于当...,本次发行的可转债给予原股东优先配售权。具体优先配售数量由股东大会授权董事会或其他被授权人士在...,NaN,NaN,NaN,NaN,"优先配售,网上定价和网下配售",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VBgIsEArxM,100000.00,1000.0000,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,6.0,NaN,NaN,Y1,NaN,本次发行的可转债的票面利率的确定方式及每一计息年度的最终利率水平，由股东大会授权公司董事会或...,NaN,NaN,20180829.0,2019-03-23 01:19:41,0
1,{00D5B779-50F9-11E7-B368-448A5B7647D3},NaN,CNY,20180222,20170615.0,20170701.0,NaN,NaN,12,0.0,NaN,NaN,NaN,NaN,NaN,NaN,本次发行可转债的初始转股价格不低于募集说明书公告日前20个交易日公司股票交易均价（若在该20...,在本次可转债发行之后，当公司发生送股、转增股本、增发新股（不包括因本次发行的可转债转股增加的...,本次可转债转股期自可转债发行结束之日满6个月后的第一个交易日起至可转债到期日止。,NaN,NaN,在本次可转债最后两个计息年度内，如果公司股票收盘价在任何连续30个交易日低于当期转股价格的7...,1.到期赎回条款在本次发行的可转债期满后5个交易日内，本公司将赎回全部未转股的可转债。具体赎...,在本次可转债存续期间，当公司股票出现在任意连续30个交易日中至少15个交易日的收盘价不高于当...,本次可转债可向公司原 A 股股东优先配售,NaN,NaN,NaN,NaN,"优先配售,网上定价和网下配售",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,uUesHYZdkg,40002.84,400.0284,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,6.0,NaN,NaN,Y1,NaN,本次发行的可转债的票面利率的确定方式及每一计息年度的最终利率水平，提请公司股东大会授权公司董...,NaN,NaN,20170615.0,2019-03-23 01:19:41,0
2,{010080EE-B77C-11E9-A242-6C0B84A68BD4},128101.SZ,CNY,20200410,20190806.0,20190823.0,20200410.0,20200413.0,3,0.0,NaN,兴业证券股份有限公司,0.0,1.0,2.5167,在本次发行的可转债期满后5个交易日内，公司将以本次发行的可转债的票面面值的110%（含最后一...,本次发行的可转债初始转股价格为18.82元/股,在本次发行之后，当公司发生派送股票股利、转增股本、增发新股或配股、派送现金股利等情况（不包括...,可转换公司债券转股期的起止日期：2020年9月21日至2026年3月16日,18.82,NaN,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于...,转股期内，当下述两种情形的任意一种出现时，公司有权决定按照债券面值加当期应计利息的价格赎回全...,在本次发行的可转换公司债券存续期间，当公司股票在任意连续三十个交易日中至少有十五个交易日的收...,原股东可优先配售的联创转债数量为其在股权登记日（2020年3月13日，T-1日）收市后持有的...,20191227.0,20200303.0,20200312.0,20200320.0,优先配售和上网定价,5858490.56,20200316.0,20200313.0,20200316.0,82036.0,联创配债,100.0,1.0,0.4208,761430.0,NaN,20200316.0,72036.0,联创发债,100.0,2204807.0,NaN,19859.2352,NaN,4.378578e+09,NaN,NaN,0.005035,NaN,NaN,NaN,NaN,1AE4F2BC75,30000.00,300.0000,S10983966,NaN,NaN,NaN,NaN,1,1000.0,100.0,6.0,累进利率,0.3,Y1,0.005113,本次发行的可转债票面利率具体为：第一年0.30%、第二年0.50%、第三年1.00%、第四年...,20200317.0,20200318.0,20190806.0,2020-04-10 10

**Text feature description(sample):**
+ CB_CLAUSE_COMINTERESTITEM: 到期赎回条款 (在本次发行的可转债期满后五个交易日内，公司将以本次可转债票面面值的111%（含最后一期利息）的价格赎回全部未转股的可转债)
+ CB_CLAUSE_CONVERSIONITEM: 发行时的转股条款(本次发行的可转债初始转股价格为18.82元/股)
+ CB_CLAUSE_CONVCHANGEITEM: 转股价格修正条款(分红/配股等导致股价变化时转股价也按比例变化)
+ CB_CLAUSE_CONVMONTH: 转股期条款(本次发行的可转债转股期自可转债发行结束之日满六个月后的第一个交易日起至可转债到期日止)
+ CB_CLAUSE_CALLITEM: 回售条款(在本次发行的可转换公司债券最后两个计息年度内，如果公司股票任何连续三十个交易日的收盘价格低于当期转股价格的70%时，可转换公司债券持有人有权将其持有的可转换公司债券全部或部分按债券面值加当期应计利息的价格回售给公司)
+ CB_CLAUSE_RESET_ITEM: 强赎条款(转股期内，如果公司股票连续三十个交易日中至少有十五个交易日的收盘价不低于当期转股价格的130%（含130%），或本次发行的可转债未转股余额不足人民币3,000万元时，公司董事会有权决定按照债券面值加当期应计利息的价格赎回全部或部分未转股的可转债)
+ CB_CLAUSE_RESETITEM: 下修条款(当公司股票在任意连续二十个交易日中至少有十个交易日的收盘价低于当期转股价格的90%时，公司董事会有权提出转股价格向下修正方案并提交公司股东大会审议表决。须三分之二以上通过方可实施,修正后的转股价格应不低于前述的股东大会召开日前二十个交易日公司股票交易均价和前一个交易日公司股票交易均价)
+ CB_CLAUSE_RATIONITEM: 股东优先配售条款(本次发行的可转债给予原股东优先配售权)
+ B_INFO_INTERESTTYPE: 利率类型(累进利率/固定利率)
+ B_INFO_COUPONRATE: 债券票面利率
+ B_INFO_INTERESTFREQUENCY: 付息周期
+ B_INFO_COUPONTXT: 具体的利率(票面利率第一年0.30%、第二年0.50%、第三年1.00%、第四年1.50%、第五年1.80%、第六年2.00%)

In [10]:
issuance_data = issuance_raw[['S_INFO_COMPCODE', 'S_INFO_WINDCODE', 'B_INFO_TERM_YEAR_', 
                              'IS_CONVERTIBLE_BONDS', 'CB_INFO_LISTEDDATE',
                              'CB_CLAUSE_COMINTERESTITEM', 'CB_CLAUSE_CALLITEM', 
                             'CB_CLAUSE_RESET_ITEM','B_INFO_INTERESTFREQUENCY']].copy()

issuance_data.rename(columns = {'S_INFO_COMPCODE': 'comp_code',
                                'S_INFO_WINDCODE': 'bond_ticker',
                                'IS_CONVERTIBLE_BONDS': 'is_conv',
                                'B_INFO_TERM_YEAR_':'term_year',
                                'CB_INFO_LISTEDDATE':'listed_date',
                                'CB_CLAUSE_COMINTERESTITEM': 'issue_item',
                               'CB_CLAUSE_CALLITEM':'put_item',
                               'CB_CLAUSE_RESET_ITEM': 'call_item', 
                               'B_INFO_INTERESTFREQUENCY': 'interest_freq'}, inplace=True)

# only select convertible bond and ticker_length==9 (e.g. 128101.SZ, listed convertible bond)
issuance_data['ticker_length'] = issuance_data['bond_ticker'].map(lambda x: len(x) if x is not np.nan else 0)
issuance_data = issuance_data[(issuance_data['is_conv'] == 1)
                              & (issuance_data['ticker_length'] == 9)].reset_index(drop=True).copy()
issuance_data.drop(columns=['is_conv', 'ticker_length'], inplace=True)
issuance_data.head(1)

,comp_code,bond_ticker,term_year,listed_date,issue_item,put_item,call_item,interest_freq
0,1AE4F2BC75,128101.SZ,6.0,20200413.0,在本次发行的可转债期满后5个交易日内，公司将以本次发行的可转债的票面面值的110%（含最后一...,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于...,转股期内，当下述两种情形的任意一种出现时，公司有权决定按照债券面值加当期应计利息的价格赎回全...,Y1


Note: `comp_code` is used to match `stock_ticker` and `bond_ticker`

In [11]:
issuance_data.to_csv(os.path.join(output_dir, 'issuance_data.csv'), index=False)

+ Redemption (强制赎回条款)

In [12]:
redemption_raw = pd.read_csv(os.path.join(data_dir, 'data', cbond_file_map['redemption']), delimiter="|")
redemption_raw.head(3)

,OBJECT_ID,B_INFO_WINDCODE,B_INFO_REDEMPTIONPRICE,B_INFO_BGNDT,B_INFO_ENDDT,B_INFO_TRNSRT,OPDATE,OPMODE
0,{000596F5-33C7-11E8-AA06-6C0B8445CDD8},137063.SH,NaN,20181008.0,20210404,130.0,2018-04-10 02:04:31,0
1,{001E5FD5-2E35-11EA-B219-6C0B84A6D7E1},120004.SZ,NaN,20210826.0,20250108,130.0,2020-01-04 13:13:03,0
2,{00587DFE-9395-11E7-B09C-6C0B84A6D803},117094.SZ,NaN,20180308.0,20200817,130.0,2017-09-08 02:35:51,0


In [13]:
redemption_data = redemption_raw[['B_INFO_WINDCODE', 'B_INFO_REDEMPTIONPRICE', 'B_INFO_BGNDT', 
                                'B_INFO_ENDDT', 'B_INFO_TRNSRT']].copy()

redemption_data.rename(columns = {'B_INFO_WINDCODE': 'bond_ticker',
                                 'B_INFO_REDEMPTIONPRICE': 'rdmpt_price',
                                 'B_INFO_BGNDT':'rdmpt_bgndt',
                                 'B_INFO_ENDDT':'rdmpt_enddt',
                                 'B_INFO_TRNSRT':'rdmpt_transit'}, inplace=True)
redemption_data.head()

,bond_ticker,rdmpt_price,rdmpt_bgndt,rdmpt_enddt,rdmpt_transit
0,137063.SH,NaN,20181008.0,20210404,130.0
1,120004.SZ,NaN,20210826.0,20250108,130.0
2,117094.SZ,NaN,20180308.0,20200817,130.0
3,127017.SZ,NaN,20201209.0,20260602,130.0
4,128069.SZ,NaN,20191230.0,20250624,130.0


In [14]:
# save redemption data to csv
redemption_data.to_csv(os.path.join(output_dir, 'redemption_data.csv'), index=False)

+ Repurchase (回售条款)

In [15]:
repurchase_raw = pd.read_csv(os.path.join(data_dir, 'data', cbond_file_map['repurchase']), delimiter="|")
repurchase_raw.head(3)

,OBJECT_ID,B_INFO_WINDCODE,B_INFO_REPURCHASEPRICE,B_INFO_BGNDT,B_INFO_ENDDT,B_INFO_TRNSRT,OPDATE,OPMODE
0,{0027F6A5-F287-49F6-8839-410B3BBA830B},125729.SZ,103.0,20051016,20061015,70.0,2005-09-16 03:34:02,0
1,{00B9BB1C-CF2E-11E7-8400-6C0B84A6D803},123002.SZ,NaN,20211124,20231124,70.0,2017-11-23 01:07:55,0
2,{00E4183A-8D82-11E7-9225-842B2B44151B},132012.SH,NaN,20200304,20200904,70.0,2017-08-31 21:06:10,0


In [16]:
repurchase_data = repurchase_raw[['B_INFO_WINDCODE', 'B_INFO_REPURCHASEPRICE', 'B_INFO_BGNDT', 
                                'B_INFO_ENDDT', 'B_INFO_TRNSRT']].copy()

repurchase_data.rename(columns = {'B_INFO_WINDCODE': 'bond_ticker',
                                 'B_INFO_REPURCHASEPRICE': 'rpchs_price',
                                 'B_INFO_BGNDT':'rpchs_bgndt',
                                 'B_INFO_ENDDT':'rpchs_enddt',
                                 'B_INFO_TRNSRT':'rpchs_transit'}, inplace=True)
repurchase_data.head()

,bond_ticker,rpchs_price,rpchs_bgndt,rpchs_enddt,rpchs_transit
0,125729.SZ,103.0,20051016,20061015,70.0
1,123002.SZ,NaN,20211124,20231124,70.0
2,132012.SH,NaN,20200304,20200904,70.0
3,110019.SH,105.0,20130323,20170323,70.0
4,117171.SZ,NaN,20210219,20230817,80.0


In [17]:
repurchase_data.to_csv(os.path.join(output_dir, 'repurchase_data.csv'), index=False)

+ Convert Price

In [18]:
convprice_raw = pd.read_csv(os.path.join(data_dir, 'data', cbond_file_map['convprice']), delimiter="|")
convprice_raw.head(3)

,OBJECT_ID,S_INFO_WINDCODE,S_INFO_ENDDATE,B_INFO_ANNOUNCEMENTDATE,CB_ANAL_CONVPRICE,B_INFO_CHANGEREASON,OPDATE,OPMODE
0,{0025FCAB-B702-11E8-8A42-C03FD57AD177},117100.SZ,20180913,20180913,11.000,9.0,2018-09-14 07:32:30,0
1,{002F7935-967D-11E9-A2DB-6C0B84A68BD4},128063.SZ,20190626,20190625,8.670,10.0,2019-06-25 21:01:36,0
2,{005ACDF9-A225-11E9-A46F-6C0B8498234E},117074.SZ,20190712,20190709,13.898,3.0,2019-07-10 06:04:14,0


In [19]:
convprice_data = convprice_raw[['S_INFO_WINDCODE', 'S_INFO_ENDDATE', 'CB_ANAL_CONVPRICE']].copy()

convprice_data.rename(columns = {'S_INFO_WINDCODE': 'bond_ticker',
                                 'S_INFO_ENDDATE': 'conv_date',
                                 'CB_ANAL_CONVPRICE':'conv_price'}, inplace=True)
convprice_data.head()

,bond_ticker,conv_date,conv_price
0,117100.SZ,20180913,11.000
1,128063.SZ,20190626,8.670
2,117074.SZ,20190712,13.898
3,113021.SH,20200715,6.980
4,123018.SZ,20200716,8.290


In [20]:
# check nan data
convprice_data.isnull().sum().sum()

0

In [21]:
convprice_data.to_csv(os.path.join(output_dir, 'convprice_data.csv'), index=False)

+ Convertible bond price

In [22]:
price_raw = pd.read_csv(os.path.join(data_dir, 'data', cbond_file_map['price']), delimiter="|")
price_raw.head(3)

,OBJECT_ID,S_INFO_WINDCODE,TRADE_DT,B_DQ_OPEN,B_DQ_HIGH,B_DQ_LOW,B_DQ_ORIGINCLOSE,B_DQ_VOLUME,B_DQ_AMOUNT,OPDATE,OPMODE
0,{314CD02F-FCAB-11E9-BD4D-448A5B4D64D9},136807.SH,20191030,106.490000,106.490000,106.490000,106.490000,54800.0,58356.52,2019-11-02 09:28:38,0
1,{452CD4DD-4554-C7D3-E053-1001C80A2702},122315.SH,20170103,106.877260,106.877260,106.877260,106.877260,0.0,0.00,2017-01-04 05:00:00,0
2,{452CD4DD-4556-C7D3-E053-1001C80A2702},122321.SH,20170103,106.075068,106.075068,106.075068,106.075068,0.0,0.00,2017-01-04 05:00:01,0


In [23]:
price_data = price_raw[['S_INFO_WINDCODE', 'TRADE_DT', 'B_DQ_OPEN', 'B_DQ_HIGH', 'B_DQ_LOW','B_DQ_ORIGINCLOSE',
                                'B_DQ_VOLUME', 'B_DQ_AMOUNT']].copy()

price_data.rename(columns = {'S_INFO_WINDCODE': 'bond_ticker',
                             'TRADE_DT': 'date',
                             'B_DQ_OPEN':'open',
                             'B_DQ_HIGH':'high',
                             'B_DQ_LOW':'low',
                             'B_DQ_ORIGINCLOSE':'close',
                             'B_DQ_VOLUME':'volume',
                             'B_DQ_AMOUNT':'amount'}, inplace=True)
price_data.head()

,bond_ticker,date,open,high,low,close,volume,amount
0,136807.SH,20191030,106.490000,106.490000,106.490000,106.490000,54800.0,58356.52
1,122315.SH,20170103,106.877260,106.877260,106.877260,106.877260,0.0,0.00
2,122321.SH,20170103,106.075068,106.075068,106.075068,106.075068,0.0,0.00
3,122363.SH,20170103,106.754192,106.754192,106.754192,106.754192,0.0,0.00
4,122399.SH,20170103,101.626521,101.626521,101.626521,101.626521,0.0,0.00


In [24]:
# only select convertible bonds appeared in issuance data 
cbond_tickers = issuance_data['bond_ticker'].unique().tolist()

price_data = price_data[price_data['bond_ticker'].isin(cbond_tickers)].reset_index(drop=True).copy()

# drop the data with zero trading volume(inactive bond)
price_data = price_data[price_data['volume'] != 0].copy()

price_data['date'] = pd.to_datetime(price_data['date'].astype('str'))
price_data.head()

,bond_ticker,date,open,high,low,close,volume,amount
0,110033.SH,2017-01-03,114.82,115.80,114.820,115.320,3046.0,3513.594
1,110035.SH,2017-01-03,124.47,125.70,124.440,125.210,22842.0,28672.482
3,128011.SZ,2017-01-03,128.00,128.78,127.654,127.654,897.0,1149.171
4,110031.SH,2017-01-03,108.79,109.38,108.400,108.500,7584.0,8258.409
5,113010.SH,2017-01-03,114.77,115.10,114.400,114.800,5359.0,6159.205


In [25]:
price_data.isnull().sum().sum()

0

Now we have 6 DataFrame:
+ comp_info
+ issuance_data
+ redemption_data
+ repurchase_data
+ convprice_data(replaced by sina cvprice later, more completed)
+ price_data

Next we will combine these seperate tables into one.

In [26]:
# merge issuance data and comp info
cbond_info = pd.merge(left=issuance_data, 
                      right=comp_info[['comp_code', 'stock_ticker', 'name']],
                      on='comp_code', how='left')

In [27]:
# 6 bond_ticker can not found corresponding stock_ticker
len(cbond_info[cbond_info['stock_ticker'].isnull()])

6

In [28]:
# dropna on bond_ticker and stock_ticker
cbond_info = cbond_info.dropna(subset=['bond_ticker', 'stock_ticker']).reset_index(drop=True).copy()
cbond_info.head(1)

,comp_code,bond_ticker,term_year,listed_date,issue_item,put_item,call_item,interest_freq,stock_ticker,name
0,1AE4F2BC75,128101.SZ,6.0,20200413.0,在本次发行的可转债期满后5个交易日内，公司将以本次发行的可转债的票面面值的110%（含最后一...,在本次发行的可转换公司债券最后两个计息年度，如果公司股票在任何连续三十个交易日的收盘价格低于...,转股期内，当下述两种情形的任意一种出现时，公司有权决定按照债券面值加当期应计利息的价格赎回全...,Y1,002036.SZ,联创电子


In [29]:
# merge redemption data
cbond_info = pd.merge(left=cbond_info, 
                      right=redemption_data[['bond_ticker', 'rdmpt_bgndt', 'rdmpt_enddt', 'rdmpt_transit']],
                      on='bond_ticker', how='left')

In [30]:
# merge repurchase data
cbond_info = pd.merge(left=cbond_info, 
                      right=repurchase_data[['bond_ticker', 'rpchs_bgndt', 'rpchs_enddt', 'rpchs_transit']],
                      on='bond_ticker', how='left')

In [31]:
# load the convertible price crawled from sina finance, which is more completed then local dataset
sina_cvprice = pd.read_csv(os.path.join(output_dir, 'sina_cvprice.csv'), index_col=False)

In [32]:
sina_cvprice.head(3)

,bond_ticker,announce_date,conv_date,conv_price,conv_ratio
0,110033.SH,20210625,20210705,6.72,14.881
1,110033.SH,20210119,20210121,7.18,13.928
2,110033.SH,20200623,20200703,7.19,13.908


In [33]:
# merge convprice data
cbond_info = pd.merge(left=cbond_info, 
                      right=sina_cvprice[['bond_ticker', 'conv_date', 'conv_price']],
                      on='bond_ticker', how='left')

In [34]:
# get Start date of each bond using bond price dataset
def getStartdate(bond_ticker):
    bond_data = price_data[(price_data['bond_ticker']==bond_ticker)]
    if len(bond_data) == 0:
        return np.nan
    
    start_date = bond_data['date'].iloc[0]
    return start_date

# get Exit date of each bond using bond price dataset
def getExitdate(bond_ticker):
    bond_data = price_data[(price_data['bond_ticker']==bond_ticker)]
    if len(bond_data) == 0:
        return np.nan
    
    last_date = bond_data['date'].iloc[-1]
    return last_date

# generate column 'start_date' and 'exit_date' using bond price data
# 'start_date' and 'exit_date' represent valid tradable date range for each convertible bond
cbond_info['start_date'] = cbond_info['bond_ticker'].map(getStartdate)
cbond_info['exit_date'] = cbond_info['bond_ticker'].map(getExitdate)

# dropna on necessary columns
cbond_info.dropna(subset=['conv_date', 'rdmpt_bgndt', 'rdmpt_enddt', 'start_date', 'exit_date'], inplace=True)

# data type transformation
cbond_info['conv_date'] = pd.to_datetime(cbond_info['conv_date'].astype('int64').astype('str'))
cbond_info['rdmpt_bgndt'] = pd.to_datetime(cbond_info['rdmpt_bgndt'].astype('int64').astype('str'))
cbond_info['rdmpt_enddt'] = pd.to_datetime(cbond_info['rdmpt_enddt'].astype('int64').astype('str'))
cbond_info['rpchs_bgndt'] = cbond_info['rpchs_bgndt'].map(lambda x: pd.to_datetime(str(int(x))) if not math.isnan(x) else x)
cbond_info['rpchs_enddt'] = cbond_info['rpchs_enddt'].map(lambda x: pd.to_datetime(str(int(x))) if not math.isnan(x) else x)
cbond_info['listed_date'] = cbond_info['listed_date'].map(lambda x: pd.to_datetime(str(int(x))) if not math.isnan(x) else x)

# ============ text feature processing ===========

# extract payment infomation included in issue_item context
cbond_info['issue_num'] = cbond_info['issue_item'].map(lambda x: re.findall('\d+\.?\d*', x))
cbond_info['payment'] = cbond_info['issue_num'].map(lambda x: np.nan if len(x)==0 else 
                                                    (float(x[-1]) if float(x[-1]) > 100 else float(x[-1]) + 100))

# special case: index 35 to 41 used chinese characters to represent '106', we manually set the payment here
cbond_info.loc[cbond_info['payment'].isna(), 'payment'] = 106
cbond_info.drop(columns=['issue_item', 'issue_num'], inplace=True)

# interest payment frequency 
cbond_info['interest_freq'] = cbond_info['interest_freq'].map(lambda x: re.findall('\d+\.?\d*', x)[-1])

# put item
cbond_info['put_num'] = cbond_info['put_item'].map(lambda x: re.findall('\d+\.?\d*', x) if x is not np.nan else [])
cbond_info['put_win'] = 30
# some put prices are set to be 103(fixed),　others are set to be 100 plus accured interest. IA=B×i×t/365
cbond_info['put_price'] = cbond_info['put_num'].map(lambda x: 103 if '103' in x else 100)
cbond_info.drop(columns=['put_item', 'put_num'], inplace=True)

# call item most of them is '15/30', which means the callability will trigger
# if the convertible price larger than rdmpt_transit at least 15 days in the past 30 days
# the call price is 100 plus accured interest
cbond_info['call_win'] = 30
cbond_info['call_min'] = 15
cbond_info['call_price'] = 100
cbond_info.drop(columns=['call_item'], inplace=True)

cbond_info = cbond_info.sort_values(by=['bond_ticker', 'conv_date']).reset_index(drop=True)

cbond_cols = ['bond_ticker', 'stock_ticker', 'name', 'listed_date', 'start_date', 'exit_date', 
              'conv_date', 'conv_price', 'rdmpt_bgndt', 'rdmpt_enddt', 'rdmpt_transit', 
              'rpchs_bgndt', 'rpchs_enddt', 'rpchs_transit', 'term_year', 'interest_freq', 'payment',
             'put_win', 'put_price', 'call_win', 'call_min', 'call_price']

cbond_info = cbond_info[cbond_cols].copy()
cbond_info.head()

,bond_ticker,stock_ticker,name,listed_date,start_date,exit_date,conv_date,conv_price,rdmpt_bgndt,rdmpt_enddt,rdmpt_transit,rpchs_bgndt,rpchs_enddt,rpchs_transit,term_year,interest_freq,payment,put_win,put_price,call_win,call_min,call_price
0,110030.SH,600185.SH,格力地产,2015-01-13,2017-01-03,2019-12-10,2014-12-23,20.90,2015-06-30,2019-12-24,130.0,2016-12-25,2019-12-25,70.0,5.0,1,106.0,30,103,30,15,100
1,110030.SH,600185.SH,格力地产,2015-01-13,2017-01-03,2019-12-10,2016-05-26,7.39,2015-06-30,2019-12-24,130.0,2016-12-25,2019-12-25,70.0,5.0,1,106.0,30,103,30,15,100
2,110030.SH,600185.SH,格力地产,2015-01-13,2017-01-03,2019-12-10,2016-08-25,7.26,2015-06-30,2019-12-24,130.0,2016-12-25,2019-12-25,70.0,5.0,1,106.0,30,103,30,15,100
3,110030.SH,600185.SH,格力地产,2015-01-13,2017-01-03,2019-12-10,2017-07-11,7.24,2015-06-30,2019-12-24,130.0,2016-12-25,2019-12-25,70.0,5.0,1,106.0,30,103,30,15,100
4,110030.SH,600185.SH,格力地产,2015-01-13,2017-01-03,2019-12-10,2018-08-20,6.94,2015-06-30,2019-12-24,130.0,2016-12-25,2019-12-25,70.0,5.0,1,106.0,30,103,30,15,100


In [35]:
# 359 valid convertible bonds in total
bond_tickers = cbond_info['bond_ticker'].unique().tolist()
len(bond_tickers)

359

In [36]:
# save the whole table to csv
cbond_info.to_csv(os.path.join(output_dir, 'cbond_info.csv'), index=False)

In [37]:
# select the 359 convertible bond price data
price_data = price_data[price_data['bond_ticker'].isin(bond_tickers)].reset_index(drop=True).copy()

In [38]:
price_data['pre_close'] = price_data.groupby('bond_ticker')['close'].shift()
price_data['ovnt_ret'] = price_data['open'] / price_data['pre_close'] - 1
price_data['intra_ret'] = price_data['close'] / price_data['open'] - 1
price_data['return'] = price_data['close'] / price_data['pre_close'] - 1
price_data['next_return'] = price_data.groupby('bond_ticker', as_index=False)['return'].shift(-1)
price_data['is_last'] = price_data.groupby('bond_ticker', as_index=False)['close'].shift(-1).isna() # is last trade day or not

In [39]:
price_data.to_csv(os.path.join(output_dir, 'cbond_price.csv'), index=False)

**Data Processing for `stock_data`**

In [40]:
sorted(os.listdir(os.path.join(data_dir, 'stock_data')))

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 'adj',
 'adj2']

In [41]:
stock_data = {}

data_columns = ['BloombergID', 'CUSIP', 'ISIN', 'ticker', 'open', 'close', 'high', 'low', 'volume', 
               'NA', 'NA', 'NA', 'shares', 'NA', 'currency', 'USD_currecny_rate', 'exchangeID', 'sectorID',
               'NA', 'industryID', 'subindustryID', 'volume_20MA']

keep_columns = ['BloombergID', 'stock_ticker', 'open', 'close', 'high', 'low', 'volume', 'shares']

# transfer stock ticker from int64 to string
def transTicker(ticker):
    ticker = str(ticker)
    stock_ticker = (6 - len(ticker)) * '0' + ticker
    
    if stock_ticker.startswith('6'):
        stock_ticker += '.SH'
    else:
        stock_ticker += '.SZ'
        
    return stock_ticker

# Load data
years = sorted(os.listdir(os.path.join(data_dir, 'stock_data')))[:-2]

for year in tqdm(years):
    months = sorted(os.listdir(os.path.join(data_dir, 'stock_data', year)))
    
    for month in months:
        raw_prices = sorted(os.listdir(os.path.join(data_dir, 'stock_data', year, month)))
        
        for raw_price in raw_prices:
            date = raw_price.split('.')[1]
            if raw_price.startswith('.'): # files we don't need
                continue
                
            data = pd.read_csv(os.path.join(data_dir, 'stock_data', year, month, raw_price), header=None)
            
            data.columns = data_columns
            data['stock_ticker'] = data['ticker'].map(transTicker)
            data = data[keep_columns].copy()
            stock_data[date] = data

100%|██████████| 11/11 [00:17<00:00,  1.61s/it]


In [42]:
# stock_data(dict): {date(str): daily data(DataFrame)} 
stock_data['20161230'].head() # sample

,BloombergID,stock_ticker,open,close,high,low,volume,shares
0,EQ0000000025800901,600959.SH,11.22,11.30,11.33,11.15,14209641,3884529799
1,EQ0000000001131660,002001.SZ,19.16,19.60,19.73,19.16,8794571,1088919000
2,EQ0730727200001000,600380.SH,9.95,9.93,10.04,9.92,4495748,1587139292
3,EQ0000000000871248,600340.SH,23.99,23.90,23.99,23.59,7118963,2954946709
4,EQ0000000009518485,600917.SH,13.18,13.19,13.32,13.17,4000800,1556000000


#### Adjust factor (复权因子)

In [43]:
# Load adjust factor from dataset
adj_data = []
adj_files = sorted(os.listdir(os.path.join(data_dir, 'stock_data', 'adj2')))

for file in tqdm(adj_files):
    if file.startswith('.'):
        continue
    date = file.split('.')[1]
    adj = pd.read_csv(os.path.join(data_dir, 'stock_data', 'adj2', file), sep='|', header=None)
    adj = adj.iloc[:, [0, 7, -2, -3]]
    adj.columns = ['BloombergID', 'adj_return', 'adj_factor', 'type']
    adj['date'] = date
    adj_data.append(adj)

adj_data = pd.concat(adj_data).reset_index(drop=True)

100%|██████████| 1945/1945 [00:07<00:00, 244.40it/s]


In [44]:
# Bloomberg adjusted factor data have lags
adj_data['adj_return'] = adj_data.groupby('BloombergID')['adj_return'].shift(-1)
adj_data['adj_factor'] = adj_data.groupby('BloombergID')['adj_factor'].shift(-2)
adj_data['top1500'] = adj_data['type'].map(lambda x: 'top1500' in x if x is not np.nan else False)

In [45]:
adj_data.head()

,BloombergID,adj_return,adj_factor,type,date,top1500
0,EQ0000000001131660,-0.018251,0.224794,"top1500,all",20130104,True
1,EQ0730727200001000,-0.017938,0.357461,all,20130104,False
2,EQ0000000000871248,0.005652,0.270899,"top1500,all",20130104,True
3,EQ0000000001262810,0.013712,0.947218,"top1500,all",20130104,True
4,EQ0000000000227277,-0.019231,0.894337,"top1500,all",20130104,True


#### calculate stock data features

In [46]:
# load convertible bond data from csv
cbond_data = pd.read_csv(os.path.join(output_dir, 'cbond_price.csv'), index_col=False)
cbond_data['date'] = cbond_data['date'].astype('str').map(lambda x: x.replace('-', ''))

trade_dates = list(cbond_data['date'].unique())
trade_dates[0]

'20170103'

In [49]:
# concat stock data to stock_df
stock_df = []
stock_dates = [d for d in stock_data.keys() if (d >= '20150101') and (d <= '20161230')] + trade_dates # don't change this code!

for date in stock_dates:
    df = stock_data[date]
    df['date'] = date
    stock_df.append(df)

stock_df = pd.concat(stock_df, axis=0).reset_index(drop=True)

stock_df.head()

,BloombergID,stock_ticker,open,close,high,low,volume,shares,date
0,EQ0000000001006672,600455.SH,19.20,19.10,19.60,18.71,1655798,62458000,20150101
1,EQ0000000001131660,002001.SZ,15.34,15.17,15.37,14.99,14125161,1088919000,20150101
2,EQ0000000002917154,002087.SZ,4.77,4.83,4.86,4.77,6005451,519758400,20150101
3,EQ0000000000871248,600340.SH,40.97,43.60,43.84,40.00,17985442,1322879700,20150101
4,EQ0000000009408441,300039.SZ,12.39,12.68,12.81,12.36,8062307,640376000,20150101


In [50]:
# add adjust factor
stock_df = stock_df[(stock_df['volume'] != 0) & (stock_df['shares'] > 0)].copy()
stock_df = pd.merge(left=stock_df, right=adj_data, on=['BloombergID', 'date'], how='left')

stock_df['top1500'].fillna(False, inplace=True)  # nan data replace to False
stock_df['adj_factor'] = stock_df.groupby('stock_ticker')['adj_factor'].fillna(method='bfill')
stock_df['adj_factor'] = stock_df['adj_factor'].fillna(1) # deal with nan happened at first
stock_df['adj_close'] = stock_df['close'] / stock_df['adj_factor']
stock_df['adj_open'] = stock_df['open'] / stock_df['adj_factor']
stock_df['adj_pre_close'] = stock_df.groupby('stock_ticker')['adj_close'].shift()

In [51]:
# check nan data
stock_df['adj_close'].isna().sum() + stock_df['adj_open'].isna().sum()

0

In [52]:
# caculate daily return and rolling volatility
stock_df['return'] = stock_df.groupby('stock_ticker', as_index=False)['adj_close'].pct_change()
stock_df['next_return'] = stock_df.groupby('stock_ticker', as_index=False)['return'].shift(-1)
stock_df['vol100'] = stock_df.groupby('stock_ticker', as_index=False)['return'].rolling(100, min_periods=10).std(ddof=1).sort_index()['return']
stock_df['vol250'] = stock_df.groupby('stock_ticker', as_index=False)['return'].rolling(100, min_periods=10).std(ddof=1).sort_index()['return']
stock_df = stock_df[stock_df['date'] >= '20170103'].copy() # only need data later than 20170103

In [53]:
stock_df.head()

,BloombergID,stock_ticker,open,close,high,low,volume,shares,date,adj_return,adj_factor,type,top1500,adj_close,adj_open,adj_pre_close,return,next_return,vol100,vol250
1525101,EQ0000000025800901,600959.SH,11.25,11.28,11.38,11.24,13425182,3884529799,20170103,-0.001772,0.763482,"top1500,all",True,14.774415,14.735121,14.800611,-0.001770,-0.003546,0.010620,0.010620
1525102,EQ0000000001131660,002001.SZ,19.60,19.97,20.00,19.56,10184249,1088919000,20170103,0.018701,0.138791,"top1500,all",True,143.885410,141.219532,141.219532,0.018878,-0.005508,0.016235,0.016235
1525103,EQ0730727200001000,600380.SH,9.94,10.02,10.03,9.94,5284526,1587139292,20170103,0.009023,0.345302,NaN,False,29.018077,28.786396,28.757436,0.009063,0.006986,0.023298,0.023298
1525104,EQ0000000000871248,600340.SH,23.90,23.89,23.99,23.69,6917735,2954946709,20170103,-0.000419,0.085648,"top1500,all",True,278.931723,279.048480,279.048480,-0.000418,0.012976,0.012515,0.012515
1525105,EQ0000000009518485,600917.SH,13.24,13.49,13.53,13.24,5394217,1556000000,20170103,0.022490,0.977647,"top1500,all",True,13.798436,13.542720,13.491577,0.022745,0.005189,0.033246,0.033246


In [54]:
# sample check return and next_return
stock_df[['BloombergID', 'stock_ticker', 'date', 'close', 'adj_return', 
          'adj_factor', 'return', 'next_return', 'vol100', 'vol250']].sort_values(by=['stock_ticker', 'date']).iloc[:10]

,BloombergID,stock_ticker,date,close,adj_return,adj_factor,return,next_return,vol100,vol250
1526988,EQ0013088700003000,000001.SZ,20170103,9.16,0.006572,0.206878,0.006593,0.000000,0.006438,0.006438
1529819,EQ0013088700003000,000001.SZ,20170104,9.16,0.000000,0.206878,0.000000,0.001092,0.006232,0.006232
1532652,EQ0013088700003000,000001.SZ,20170105,9.17,0.001091,0.206878,0.001092,-0.004362,0.006223,0.006223
1535490,EQ0013088700003000,000001.SZ,20170106,9.13,-0.004372,0.206878,-0.004362,0.002191,0.006217,0.006217
1538330,EQ0013088700003000,000001.SZ,20170109,9.15,0.002188,0.206878,0.002191,0.000000,0.006221,0.006221
1541170,EQ0013088700003000,000001.SZ,20170110,9.15,0.000000,0.206878,0.000000,-0.001093,0.006117,0.006117
1544013,EQ0013088700003000,000001.SZ,20170111,9.14,-0.001093,0.206878,-0.001093,0.001094,0.006117,0.006117
1546862,EQ0013088700003000,000001.SZ,20170112,9.15,0.001093,0.206878,0.001094,0.001093,0.006109,0.006109
1549715,EQ0013088700003000,000001.SZ,20170113,9.16,0.001092,0.206878,0.001093,-0.002183,0.006109,0.006109
1552564,EQ0013088700003000,000001.SZ,20170116,9.14,-0.002186,0.206878,-0.002183,0.001094,0.006111,0.006111


In [55]:
# spring festival check, should be empty DataFrame
stock_df[stock_df['date']=='20170127']

,BloombergID,stock_ticker,open,close,high,low,volume,shares,date,adj_return,adj_factor,type,top1500,adj_close,adj_open,adj_pre_close,return,next_return,vol100,vol250


#### check adj factor

In [56]:
stock_df['abs_return'] = stock_df['return'].abs()
stock_df['no_limit'] = stock_df['stock_ticker'].map(lambda x: x.startswith('688')) # 创业板上市没有涨跌停限制
stock_df['is_unusual'] = (stock_df['abs_return'] > 0.21) & (~stock_df['no_limit']) # abs_return大于20%的有问题
stock_df['error'] = stock_df['adj_return'] - stock_df['return']

In [57]:
# return of 20 samples are unusual. Set 'is_unusual==True' and we will avoid using these data later.
stock_df[stock_df['is_unusual'] == True]

,BloombergID,stock_ticker,open,close,high,low,volume,shares,date,adj_return,adj_factor,type,top1500,adj_close,adj_open,adj_pre_close,return,next_return,vol100,vol250,abs_return,no_limit,is_unusual,error
1600499,EQ0000000016294494,603838.SH,17.82,17.82,17.82,17.82,13084498,266680000,20170215,0.095310,0.498001,NaN,False,35.783061,35.783061,65.301074,-0.452030,0.099888,0.119467,0.119467,0.452030,False,True,0.547340
1792017,EQ0000000035268504,603986.SH,97.78,94.83,99.79,94.63,1027524,100000000,20170522,NaN,1.000000,NaN,False,94.830000,97.780000,195.000000,-0.513692,-0.074344,0.086123,0.086123,0.513692,False,True,NaN
1805662,EQ0000000035268606,300526.SZ,38.95,38.95,39.95,38.95,10784715,84907408,20170531,NaN,1.000000,NaN,False,38.950000,38.950000,86.700000,-0.550750,-0.099872,0.095430,0.095430,0.550750,False,True,NaN
2201282,EQ0019123700002000,600725.SH,2.85,2.99,2.99,2.85,4018528,1232470000,20171201,0.047954,0.258421,NaN,False,11.570267,11.028515,22.057031,-0.475439,0.050167,0.113078,0.113078,0.475439,False,True,0.523393
2236792,EQ0000000043757087,300612.SZ,59.70,59.70,59.70,59.70,1900,108000000,20171218,NaN,1.000000,NaN,False,59.700000,59.700000,100.000000,-0.403000,-0.100000,0.090432,0.090432,0.403000,False,True,NaN
2236865,EQ0145936500001000,000155.SZ,8.00,7.30,8.56,7.30,215640541,1270000000,20171218,NaN,0.907920,NaN,False,8.040356,8.811349,11.311569,-0.289192,-0.100000,0.043724,0.043724,0.289192,False,True,NaN
2361599,EQ0000000003875226,300110.SZ,12.39,12.39,12.39,12.39,93600,986105137,20180209,-0.243120,0.214993,"top1500,all",True,57.629783,57.629783,73.490765,-0.215823,-0.100081,0.024431,0.024431,0.215823,False,True,-0.027297
2789981,EQ0019379000001000,000629.SZ,3.70,3.84,4.41,3.70,673085384,8589746202,20180824,NaN,0.349258,NaN,False,10.994737,10.593888,8.790064,0.250814,-0.096354,0.030648,0.030648,0.250814,False,True,NaN
2868841,EQ0019335200002000,600733.SH,14.66,9.50,14.66,9.50,21787800,3355349137,20180927,-0.326430,0.249969,NaN,False,38.004713,58.647272,52.690000,-0.278711,0.100000,0.039424,0.039424,0.278711,False,True,-0.047719
3075721,EQ0000000000650570,600423.SH,4.23,4.23,4.23,4.23,712200,798695026,20181226,-0.050702,0.262155,NaN,False,16.135492,16.135492,9.212107,0.751553,-0.049645,0.078875,0.078875,0.751553,False,True,-0.802255


In [58]:
# calculate return
stock_df['open_ret'] = stock_df['adj_open'] / stock_df['adj_pre_close'] - 1
stock_df['close_ret'] = stock_df['adj_close'] / stock_df['adj_pre_close'] - 1

# estimate trading value and market value
stock_df['value'] = stock_df['volume'] * (stock_df['high'] + stock_df['low']) / 2
stock_df['mkt'] = stock_df['shares'] * (stock_df['high'] + stock_df['low']) / 2

# up / down limit
cond_1 = (stock_df['open_ret'] >= 0.099) * (~stock_df['no_limit'])  # non-STAR market　10% limit
cond_2 = (stock_df['open_ret'] >= 0.199) * (stock_df['no_limit'])   # STAR market　20% limit 
stock_df['limit_buy_open'] = cond_1 | cond_2 # limit to buy at open
cond_1 = (stock_df['open_ret'] <= -0.099) * (~stock_df['no_limit'])  # non-STAR market　10% limit
cond_2 = (stock_df['open_ret'] <= -0.199) * (stock_df['no_limit'])   # STAR market　20% limit
stock_df['limit_sell_open'] = cond_1 | cond_2 # limit to sell at open

cond_1 = (stock_df['close_ret'] >= 0.099) * (~stock_df['no_limit'])  # non-STAR market 10% limit 
cond_2 = (stock_df['close_ret'] >= 0.199) * (stock_df['no_limit'])   # STAR market　20% limit
stock_df['limit_buy_close'] = cond_1 | cond_2 # limit to sell at close
cond_1 = (stock_df['close_ret'] <= -0.099) * (~stock_df['no_limit'])  # non-STAR market　10% limit
cond_2 = (stock_df['close_ret'] <= -0.199) * (stock_df['no_limit'])   # STAR market　20% limit
stock_df['limit_sell_close'] = cond_1 | cond_2 # limit to sell at close

#### Index information(指数中的成分股)

In [59]:
# CSI300 and CSI500 file
files = []

years = sorted(os.listdir(os.path.join(data_dir, 'index_data')))
for year in years:
    months = sorted(os.listdir(os.path.join(data_dir, 'index_data', year)))
    for month in months:
        files += sorted(os.listdir(os.path.join(data_dir, 'index_data', year, month)))

CSI300 = [file.split('.')[1] for file in files if file.startswith('CSI300')]
CSI500 = [file.split('.')[1] for file in files if file.startswith('CSI500')]

In [60]:
def getCSI(date, index='CSI300'):
    '''
    return index member at given date
    '''
    if index == 'CSI300':
        last_date = [d for d in CSI300 if d < date][-1]
    else:
        last_date = [d for d in CSI500 if d < date][-1]
    
    year = last_date[:4]
    month = last_date[4:6]
    index_data = pd.read_csv(os.path.join(data_dir, 'index_data', year, month, '{}.{}'.format(index, last_date)), sep='|')
    index_data['exchange'] = index_data['交易所Exchange'].map(lambda x: '.'+ x[0] + x[-1])
    index_data['code'] = index_data['成分券代码Constituent Code'].map(lambda x: '0' * (6 - len(str(x))) + str(x))
    index_data['ticker'] = index_data['code'] + index_data['exchange']
    tickers = index_data['ticker'].unique().tolist()
    return tickers

In [61]:
# index information
data = []

for date, df in tqdm(stock_df.groupby('date')['stock_ticker']):
    df = df.to_frame()
    tickers300 = getCSI(date, index='CSI300')
    tickers500 = getCSI(date, index='CSI500')
    df['CSI300'] = df['stock_ticker'].isin(tickers300)
    df['CSI500'] = df['stock_ticker'].isin(tickers500)
    data.append(df)
    
data = pd.concat(data).reset_index(drop=True)

100%|██████████| 886/886 [00:04<00:00, 183.81it/s]


In [62]:
stock_df['CSI300'] = data['CSI300'].values
stock_df['CSI500'] = data['CSI500'].values
stock_df['CSI800'] = stock_df['CSI300'] | stock_df['CSI500'] 

In [63]:
stock_df = stock_df[['BloombergID', 'stock_ticker', 'date',
                     'adj_close', 'adj_open', 'adj_pre_close', 'adj_factor', 'vol100', 'vol250', 'value', 'mkt', 'top1500', 'CSI300', 'CSI800',
                     'limit_buy_open', 'limit_sell_open', 'limit_buy_close', 'limit_sell_close', 'is_unusual']].reset_index(drop=True).copy()

stock_df.head()

,BloombergID,stock_ticker,date,adj_close,adj_open,adj_pre_close,adj_factor,vol100,vol250,value,mkt,top1500,CSI300,CSI800,limit_buy_open,limit_sell_open,limit_buy_close,limit_sell_close,is_unusual
0,EQ0000000025800901,600959.SH,20170103,14.774415,14.735121,14.800611,0.763482,0.010620,0.010620,1.518388e+08,4.393403e+10,True,True,True,False,False,False,False,False
1,EQ0000000001131660,002001.SZ,20170103,143.885410,141.219532,141.219532,0.138791,0.016235,0.016235,2.014444e+08,2.153882e+10,True,False,True,False,False,False,False,False
2,EQ0730727200001000,600380.SH,20170103,29.018077,28.786396,28.757436,0.345302,0.023298,0.023298,5.276599e+07,1.584759e+10,False,False,True,False,False,False,False,False
3,EQ0000000000871248,600340.SH,20170103,278.931723,279.048480,279.048480,0.085648,0.012515,0.012515,1.649188e+08,7.044593e+10,True,True,True,False,False,False,False,False
4,EQ0000000009518485,600917.SH,20170103,13.798436,13.542720,13.491577,0.977647,0.033246,0.033246,7.220159e+07,2.082706e+10,True,False,True,False,False,False,False,False


In [64]:
# index statistics
stock_df[['top1500', 'CSI300', 'CSI800']].mean()

top1500    0.425019
CSI300     0.085376
CSI800     0.227147
dtype: float64

In [65]:
# nan check
stock_df.isna().sum()

BloombergID            0
stock_ticker           0
date                   0
adj_close              0
adj_open               0
adj_pre_close        928
adj_factor             0
vol100              9339
vol250              9339
value                  0
mkt                    0
top1500                0
CSI300                 0
CSI800                 0
limit_buy_open         0
limit_sell_open        0
limit_buy_close        0
limit_sell_close       0
is_unusual             0
dtype: int64

#### Transfer stock data to dict (date -> DataFrame)

In [66]:
stock_data_clean = {}
for date in tqdm(trade_dates):
    stock_data_clean[date] = stock_df.loc[stock_df['date']==date].copy()

100%|██████████| 886/886 [01:29<00:00,  9.94it/s]


In [67]:
# sample
stock_data_clean['20170126'].head()

,BloombergID,stock_ticker,date,adj_close,adj_open,adj_pre_close,adj_factor,vol100,vol250,value,mkt,top1500,CSI300,CSI800,limit_buy_open,limit_sell_open,limit_buy_close,limit_sell_close,is_unusual
48498,EQ0000000025800901,600959.SH,20170126,14.276695,14.276695,14.224304,0.763482,0.010977,0.010977,6.523202e+07,4.241907e+10,True,True,True,False,False,False,False,False
48499,EQ0000000001131660,002001.SZ,20170126,135.599571,135.527520,135.311367,0.138791,0.015308,0.015308,6.967097e+07,2.046079e+10,False,False,True,False,False,False,False,False
48500,EQ0730727200001000,600380.SH,20170126,27.599029,27.367348,27.425268,0.345302,0.017406,0.017406,4.542110e+07,1.512950e+10,False,False,True,False,False,False,False,False
48501,EQ0000000000871248,600340.SH,20170126,283.718747,280.566317,280.449560,0.085648,0.012269,0.012269,1.809542e+08,7.137674e+10,True,True,True,False,False,False,False,False
48502,EQ0000000009518485,600917.SH,20170126,12.785801,12.663057,12.642600,0.977647,0.033870,0.033870,3.379470e+07,1.935664e+10,True,False,True,False,False,False,False,False


In [68]:
pickle.dump(stock_data_clean, open(os.path.join(output_dir, 'stock_data_raw.pkl'), 'wb'))

**add features on cbond_price**

In [69]:
# read cbond_info and cbond_price from csv
cbond_info = pd.read_csv(os.path.join(output_dir, 'cbond_info.csv'), index_col=False)
cbond_price = pd.read_csv(os.path.join(output_dir, 'cbond_price.csv'), index_col=False)

cbond_info['listed_date'] = cbond_info['listed_date'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['start_date'] = cbond_info['start_date'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['exit_date'] = cbond_info['exit_date'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['conv_date'] = cbond_info['conv_date'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['rdmpt_bgndt'] = cbond_info['rdmpt_bgndt'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['rdmpt_enddt'] = cbond_info['rdmpt_enddt'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['rpchs_bgndt'] = cbond_info['rpchs_bgndt'].astype('str').map(lambda x: x.replace('-', ''))
cbond_info['rpchs_enddt'] = cbond_info['rpchs_enddt'].astype('str').map(lambda x: x.replace('-', ''))
cbond_price['date'] = cbond_price['date'].astype('str').map(lambda x: x.replace('-', ''))

In [70]:
# add convertible bond info to cbond_price DataFrame
def matchConv(data, conv_info):
    data['conv_price'] = np.nan
    data['rdmpt_bgndt'] = np.nan
    data['rdmpt_enddt'] = np.nan
    
    for i in range(len(conv_info)):
        conv_date = conv_info[i]['conv_date']
        conv_price = conv_info[i]['conv_price']
        rdmpt_bgndt = conv_info[i]['rdmpt_bgndt']
        rdmpt_enddt = conv_info[i]['rdmpt_enddt']
        
        data.loc[data['date'] >= conv_date, 'conv_price'] = conv_price
        data.loc[data['date'] >= conv_date, 'rdmpt_bgndt'] = rdmpt_bgndt
        data.loc[data['date'] >= conv_date, 'rdmpt_enddt'] = rdmpt_enddt
        
#     if data['conv_price'].isna().sum() != 0:
#         print(bond_ticker)      
    return data

In [71]:
cbond_info.head()

,bond_ticker,stock_ticker,name,listed_date,start_date,exit_date,conv_date,conv_price,rdmpt_bgndt,rdmpt_enddt,rdmpt_transit,rpchs_bgndt,rpchs_enddt,rpchs_transit,term_year,interest_freq,payment,put_win,put_price,call_win,call_min,call_price
0,110030.SH,600185.SH,格力地产,20150113,20170103,20191210,20141223,20.90,20150630,20191224,130.0,20161225,20191225,70.0,5.0,1,106.0,30,103,30,15,100
1,110030.SH,600185.SH,格力地产,20150113,20170103,20191210,20160526,7.39,20150630,20191224,130.0,20161225,20191225,70.0,5.0,1,106.0,30,103,30,15,100
2,110030.SH,600185.SH,格力地产,20150113,20170103,20191210,20160825,7.26,20150630,20191224,130.0,20161225,20191225,70.0,5.0,1,106.0,30,103,30,15,100
3,110030.SH,600185.SH,格力地产,20150113,20170103,20191210,20170711,7.24,20150630,20191224,130.0,20161225,20191225,70.0,5.0,1,106.0,30,103,30,15,100
4,110030.SH,600185.SH,格力地产,20150113,20170103,20191210,20180820,6.94,20150630,20191224,130.0,20161225,20191225,70.0,5.0,1,106.0,30,103,30,15,100


In [72]:
cbond_price.head()

,bond_ticker,date,open,high,low,close,volume,amount,pre_close,ovnt_ret,intra_ret,return,next_return,is_last
0,110033.SH,20170103,114.820,115.800,114.820,115.320,3046.0,3513.594,NaN,NaN,0.004355,NaN,0.007371,False
1,128011.SZ,20170103,128.000,128.780,127.654,127.654,897.0,1149.171,NaN,NaN,-0.002703,NaN,0.002703,False
2,110031.SH,20170103,108.790,109.380,108.400,108.500,7584.0,8258.409,NaN,NaN,-0.002666,NaN,0.001106,False
3,113010.SH,20170103,114.770,115.100,114.400,114.800,5359.0,6159.205,NaN,NaN,0.000261,NaN,0.002613,False
4,127003.SZ,20170103,114.669,114.669,113.659,113.800,5854.5,6666.972,NaN,NaN,-0.007578,NaN,0.008612,False


In [73]:
bond_tickers = cbond_price['bond_ticker'].unique().tolist()

# add convertible bond info to cbond_price DataFrame
data = []

for bond_ticker in tqdm(bond_tickers):

    dat = cbond_price[cbond_price['bond_ticker']==bond_ticker].copy()
    conv_info = cbond_info.loc[cbond_info['bond_ticker']==bond_ticker, 
                           ['conv_date', 'conv_price', 'rdmpt_bgndt', 'rdmpt_enddt']].to_dict(orient='records')
    data.append(matchConv(dat, conv_info))

cbond_price = pd.concat(data)

100%|██████████| 359/359 [00:02<00:00, 141.72it/s]


In [74]:
# add more features in conv_price 
cbond_price['date'] = cbond_price['date'].astype('str').map(lambda x: x.replace('-', ''))
cbond_price = cbond_price.sort_values(by=['date', 'bond_ticker']).reset_index(drop=True)

cbond_stock_map = dict(zip(cbond_info['bond_ticker'], cbond_info['stock_ticker']))
cbond_price['stock_ticker'] = cbond_price['bond_ticker'].map(lambda x: cbond_stock_map[x])
cbond_price = pd.merge(left=cbond_price, right=stock_df[['stock_ticker', 'date', 'adj_factor', 'top1500']], 
         on=['stock_ticker', 'date'], how='left')
cbond_price['adj_conv_price'] = cbond_price['conv_price'] / cbond_price['adj_factor']
cbond_price['adj_conv_share'] = 100 / cbond_price['adj_conv_price']

In [75]:
cbond_price.columns

Index(['bond_ticker', 'date', 'open', 'high', 'low', 'close', 'volume',
       'amount', 'pre_close', 'ovnt_ret', 'intra_ret', 'return', 'next_return',
       'is_last', 'conv_price', 'rdmpt_bgndt', 'rdmpt_enddt', 'stock_ticker',
       'adj_factor', 'top1500', 'adj_conv_price', 'adj_conv_share'],
      dtype='object')

In [76]:
cbond_price.to_csv(os.path.join(output_dir, 'cbond_price.csv'), index=False)
cbond_info.to_csv(os.path.join(output_dir, 'cbond_info.csv'), index=False)
stock_df.to_csv(os.path.join(output_dir, 'stock_price.csv'), index=False)

#### transfer DataFrame to dict of dict (faster in backtest)
+ stock_data[date][stock_ticker]
+ bond_data[date][bond_ticker]

In [77]:
# transfer code
cbond_data_raw = pd.read_csv(os.path.join(output_dir, 'cbond_price.csv'), index_col=False)
stock_data_raw = pickle.load(open(os.path.join(output_dir, 'stock_data_raw.pkl'), 'rb'))

cbond_data_raw['date'] = cbond_data_raw['date'].astype('str').map(lambda x: x.replace('-', ''))

dates = cbond_data_raw['date'].unique().tolist()

cbond_data = {}
stock_data = {}

for date in tqdm(dates):
    dat = cbond_data_raw[cbond_data_raw['date'] == date].copy()
    cbond_data[date] = dat.set_index('bond_ticker').to_dict(orient='index')
    stock_data[date] = stock_data_raw[date].set_index('stock_ticker').to_dict(orient='index') 

100%|██████████| 886/886 [00:12<00:00, 68.78it/s]


In [78]:
# sample
stock_data['20170103']['600959.SH']

{'BloombergID': 'EQ0000000025800901',
 'date': '20170103',
 'adj_close': 14.774415113912312,
 'adj_open': 14.735121456694461,
 'adj_pre_close': 14.800610885390881,
 'adj_factor': 0.763482,
 'vol100': 0.010620079514662822,
 'vol250': 0.010620079514662822,
 'value': 151838808.42000002,
 'mkt': 43934032026.69,
 'top1500': True,
 'CSI300': True,
 'CSI800': True,
 'limit_buy_open': False,
 'limit_sell_open': False,
 'limit_buy_close': False,
 'limit_sell_close': False,
 'is_unusual': False}

In [79]:
pickle.dump(cbond_data, open(os.path.join(output_dir, 'cbond_data.pkl'), 'wb'))
pickle.dump(stock_data, open(os.path.join(output_dir, 'stock_data.pkl'), 'wb'))